# Install

In [ ]:
!pip3 install soynlp

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [7]:
import sys
sys.path.append('drive/MyDrive/MemeProject/src/doc2vec/')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', 'drive/MyDrive/MemeProject/src/doc2vec/']


In [8]:
import config as cfg
from pshmodule.utils import filemanager as fm

In [9]:
df = fm.load(cfg.temp_data_colab)

extension : .pickle
Loaded 9141 records from drive/MyDrive/MemeProject/data/temp_for_doc2vec.pickle


In [10]:
df.head(10)

,u,soynlp
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반에, 서, 1등, 했어]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반에, 서, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 1등, 먹음]"
5,중간시험 내가 반 1등이야,"[중간, 시험, 내가, 반, 1등, 이야]"
6,너 멍청하다,"[너, 멍청하다]"
7,이 새끼 빡대가리잖아?,"[이, 새끼, 빡대가리잖아?]"
8,너 말귀 못 알아듣냐고,"[너, 말귀, 못, 알아, 듣냐고]"
9,병신 이해력 딸리네,"[병신, 이해력, 딸리네]"


# doc2vec - Okt

In [11]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
from gensim.test.utils import common_texts

### loss logger

In [ ]:
# class LossLogger(CallbackAny2Vec):
#     '''Output loss at each epoch'''
#     def __init__(self):
#         self.epoch = 1
#         self.losses = []

#     def on_epoch_begin(self, model):
#         print(f'Epoch: {self.epoch}', end='\t')

#     def on_epoch_end(self, model):
#         loss = model.get_latest_training_loss()
#         self.losses.append(loss)
#         print(f'  Loss: {loss}')
#         self.epoch += 1

In [ ]:
print('\tTagging data .. ')
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df.okt)]

print('\tPreparing model with the following parameters: epochs = {}, vector_size = {}, alpha = {} .. '.
      format(cfg.max_epochs, cfg.vec_size, cfg.alpha))

model = Doc2Vec(vector_size=cfg.vec_size,
                window=cfg.window,
                workers=cfg.workers,
                alpha=cfg.alpha,  # initial learning rate
                epochs=cfg.max_epochs, # epoch
                min_count=cfg.min_count,  # Ignore words having a total frequency below this
               )

model.build_vocab(tagged_data)

	Tagging data .. 
	Preparing model with the following parameters: epochs = 200, vector_size = 1024, alpha = 0.01 .. 


In [ ]:
cfg.vec_size

1024

In [ ]:
model.train(tagged_data,
            epochs=model.epochs,
            total_examples=len(tagged_data),
            #callbacks=[LossLogger()]
           )

### model save

In [ ]:
model.save(cfg.doc2vec_okt_colab)
print("Model Saved")

Model Saved


# doc2vec - Soynlp

In [12]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
from gensim.test.utils import common_texts

In [16]:
print(df.soynlp)

                                soynlp
0             [나, 중간고사, 반에, 서, 1등, 했어]
1     [중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]
2           [나, 반에, 서, 중간, 성적, 제일, 좋아]
3       [우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]
4                    [중간고사, 반, 1등, 먹음]
...                                ...
9136        [전국노래자랑에서, 우리, 이모가, 우승탔다구]
9137           [전국노래자랑에서, 막내이모가, 대상탔음]
9138        [울, 이모가, 전국노래자랑에서, 대상받았자나]
9139        [울, 막내이모가, 전국노래자랑에서, 우승자야]
9140           [전국노래자랑에서, 이모가, 1등, 했다]

[9141 rows x 1 columns]


In [22]:
print('\tTagging data .. ')
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(sum(df.soynlp.values.tolist(), []))]

print('\tPreparing model with the following parameters: epochs = {}, vector_size = {}, alpha = {} .. '.
      format(cfg.max_epochs, cfg.vec_size, cfg.alpha))

model = Doc2Vec(vector_size=cfg.vec_size,
                window=cfg.window,
                workers=cfg.workers,
                alpha=cfg.alpha,  # initial learning rate
                epochs=cfg.max_epochs, # epoch
                min_count=cfg.min_count,  # Ignore words having a total frequency below this
               )

model.build_vocab(tagged_data)

	Tagging data .. 
	Preparing model with the following parameters: epochs = 200, vector_size = 1024, alpha = 0.01 .. 


In [23]:
model.train(tagged_data,
            epochs=model.epochs,
            total_examples=len(tagged_data),
           )

In [24]:
index = 0
print(df.u.iloc[index])
print("-"*100)
result = model.docvecs.most_similar(index)
print(result)

u    나 중간고사 반에서 1등했어
Name: 0, dtype: object
----------------------------------------------------------------------------------------------------
[('3550', 0.8649837970733643), ('3551', 0.7612936496734619), ('3552', 0.7251943945884705), ('4', 0.6709202527999878), ('5927', 0.6299764513969421), ('9140', 0.6233336925506592), ('2139', 0.610210120677948), ('5467', 0.591822624206543), ('938', 0.5918172001838684), ('5225', 0.5894796848297119)]


In [25]:
model.save(cfg.doc2vec_soynlp_colab)
print("Model Saved")

Model Saved
